# Purpose
This notebook combines the features used in the previous 3 notebooks into a dataframe ready for modelling

### Result
This notebook will create a dataframe that will have a row for each bout and will include:
 - Features:
     - career average significant strike attempts per minute for each fighter (ca_assa_m)
     - career average significant strike defense (ca_ASSDe)
     - career average significant strike differential (ca_ASSDi)
     - 3-fight-average significant strike attempts per minute for each fighter (ca_assa_m)
     - 3-fight-average significant strike defense (fa3_ASSDe)
     - 3-fight-average significant strike differential (fa3_ASSDi)

 - Target:
     - Combined Average Significant Strike Attempts per Minute (CASSA_M)
 

In [57]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from sqlalchemy import create_engine
from src import local
from src import functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
# Credentials
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')

# Get data from postgres database
### Join tables

- get the date from the events table
- use the bouts table to join the dates to the general table
- use the general table to join the bouts with the fighters


#### Accuracy case statement

In [59]:
query = """
SELECT CONCAT(bout_link, fighter_link, CAST(round AS CHAR))
FROM general_cleaned
"""

data = pd.read_sql(query, engine)

In [60]:
data

,concat
0,http://www.ufcstats.com/fight-details/11f715fa...
1,http://www.ufcstats.com/fight-details/11f715fa...
2,http://www.ufcstats.com/fight-details/11f715fa...
3,http://www.ufcstats.com/fight-details/11f715fa...
4,http://www.ufcstats.com/fight-details/11f715fa...
...,...
26239,http://www.ufcstats.com/fight-details/cecdc0da...
26240,http://www.ufcstats.com/fight-details/2d2bbc86...
26241,http://www.ufcstats.com/fight-details/2d2bbc86...
26242,http://www.ufcstats.com/fight-details/567a09fd...


In [61]:
query = """
SELECT distance_ss_stats.bout_link, distance_ss_stats.fighter_link, distance_ss_stats.round, "Date", 
sig_str_a_p1m, td_s_p15m_di, sig_str_s_p1m_di, sig_str_a_p1m_di, 
ground_s_p15m_di, ground_a_p15m_di, clinch_s_p15m_di,
clinch_a_p15m_di, distance_a_p1m_di, distance_s_p1m_di
FROM distance_ss_stats
    JOIN clinch_ss_stats ON 
    CONCAT(distance_ss_stats.bout_link, distance_ss_stats.fighter_link, CAST(distance_ss_stats.round AS CHAR)) =
    CONCAT(clinch_ss_stats.bout_link, clinch_ss_stats.fighter_link, CAST(clinch_ss_stats.round AS CHAR))

    JOIN ground_ss_stats ON 
    CONCAT(distance_ss_stats.bout_link, distance_ss_stats.fighter_link, CAST(distance_ss_stats.round AS CHAR)) =
    CONCAT(ground_ss_stats.bout_link, ground_ss_stats.fighter_link, CAST(ground_ss_stats.round AS CHAR))

    JOIN sig_str_stats ON 
    CONCAT(distance_ss_stats.bout_link, distance_ss_stats.fighter_link, CAST(distance_ss_stats.round AS CHAR)) =
    CONCAT(sig_str_stats.bout_link, sig_str_stats.fighter_link, CAST(sig_str_stats.round AS CHAR))
    
    JOIN takedown_stats ON 
    CONCAT(distance_ss_stats.bout_link, distance_ss_stats.fighter_link, CAST(distance_ss_stats.round AS CHAR)) =
    CONCAT(takedown_stats.bout_link, takedown_stats.fighter_link, CAST(takedown_stats.round AS CHAR))

JOIN bouts ON bouts.link = distance_ss_stats.bout_link
JOIN events ON events.link = bouts.event_link
"""

data = pd.read_sql(query, engine)

In [62]:
data = functions.format_data(data, event=False)

In [63]:
data

,bout_link,fighter_link,round,Date,sig_str_a_p1m,td_s_p15m_di,sig_str_s_p1m_di,sig_str_a_p1m_di,ground_s_p15m_di,ground_a_p15m_di,clinch_s_p15m_di,clinch_a_p15m_di,distance_a_p1m_di,distance_s_p1m_di,date,fighter_id,bout_id
0,http://www.ufcstats.com/fight-details/000da315...,http://www.ufcstats.com/fighter-details/6da991...,1,"July 08, 2006",7.8,3.0,1.4,5.6,0.0,3.0,9.0,27.0,3.6,0.8,2006-07-08,6da99156486ed6c2,000da3152b7b5ab1
1,http://www.ufcstats.com/fight-details/000da315...,http://www.ufcstats.com/fighter-details/6da991...,2,"July 08, 2006",5.2,0.0,1.2,1.4,-12.0,-18.0,21.0,27.0,0.8,0.6,2006-07-08,6da99156486ed6c2,000da3152b7b5ab1
2,http://www.ufcstats.com/fight-details/000da315...,http://www.ufcstats.com/fighter-details/6da991...,3,"July 08, 2006",4.6,6.0,0.6,2.8,6.0,18.0,3.0,3.0,1.4,0.0,2006-07-08,6da99156486ed6c2,000da3152b7b5ab1
3,http://www.ufcstats.com/fight-details/000da315...,http://www.ufcstats.com/fighter-details/d1a131...,1,"July 08, 2006",2.2,-3.0,-1.4,-5.6,0.0,-3.0,-9.0,-27.0,-3.6,-0.8,2006-07-08,d1a1314976c50bef,000da3152b7b5ab1
4,http://www.ufcstats.com/fight-details/000da315...,http://www.ufcstats.com/fighter-details/d1a131...,2,"July 08, 2006",3.8,0.0,-1.2,-1.4,12.0,18.0,-21.0,-27.0,-0.8,-0.6,2006-07-08,d1a1314976c50bef,000da3152b7b5ab1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25825,http://www.ufcstats.com/fight-details/fffa2138...,http://www.ufcstats.com/fighter-details/5d7bda...,2,"October 19, 2013",3.6,6.0,0.2,0.8,-3.0,0.0,6.0,12.0,0.0,0.0,2013-10-19,5d7bdab5e03e3216,fffa21388cdd78b7
25826,http://www.ufcstats.com/fight-details/fffa2138...,http://www.ufcstats.com/fighter-details/5d7bda...,3,"October 19, 2013",8.2,3.0,1.0,3.0,6.0,9.0,3.0,0.0,2.4,0.4,2013-10-19,5d7bdab5e03e3216,fffa21388cdd78b7
25827,http://www.ufcstats.com/fight-details/fffa2138...,http://www.ufcstats.com/fighter-details/c80095...,1,"October 19, 2013",7.2,0.0,-2.0,-4.6,0.0,0.0,-12.0,-24.0,-3.0,-1.2,2013-10-19,c80095f6092271a7,fffa21388cdd78b7
25828,http://www.ufcstats.com/fight-details/fffa2138...,http://www.ufcstats.com/fighter-details/c80095...,2,"October 19, 2013",2.8,-6.0,-0.2,-0.8,3.0,0.0,-6.0,-12.0,0.0,0.0,2013-10-19,c80095f6092271a7,fffa21388cdd78b7


## Create fighter-bout instance dataframe

A fighter-bout instance represents one fighter in one bout.
 - The same fighter has exactly one fighter-bout instance for every single bout he has been in. 
 - Every bout has exactly two fighter-bout instances, one for each fighter in the bout. 
  
In this case a fighter-bout instance is assigned a unique identifier comprised of the bout_id combined with the fighter_link.

In [64]:
def create_fighter_bout_instance_table(data):
    """
    input: dataframe, a formatted fighter round instance table, either general stats or strike stats.
    output: new fighter bout instance dataframe 

    A fighter-bout instance represents one fighter in one bout.
     - The same fighter has exactly one fighter-bout instance for every single bout he has been in. 
     - Every bout has exactly two fighter-bout instances, one for each fighter in the bout. 
    In this case a fighter-bout instance is assigned a unique identifier comprised of the bout_id combined with the fighter_link.
    """
    
    data['fighter_bout_inst'] = data['bout_id'] + data['fighter_id']
    fighter_bout_inst_group = data.groupby(['fighter_bout_inst'])

    target = fighter_bout_inst_group['sig_str_a_p1m'].mean()
    date = fighter_bout_inst_group['date'].max()
    fighter_id = fighter_bout_inst_group['fighter_id'].max()
    bout_id = fighter_bout_inst_group['bout_id'].max()

    fighter_bout_inst = pd.DataFrame(dict(bout_id = bout_id, fighter_id = fighter_id, date = date, sig_str_a_p1m = target))
    return fighter_bout_inst

In [65]:
fighter_bout_inst = create_fighter_bout_instance_table(data)

In [66]:
fighter_bout_inst

,bout_id,fighter_id,date,sig_str_a_p1m
fighter_bout_inst,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,5.866667
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,2.600000
0019ec81fd706ade326f94d6cfb1bf25,0019ec81fd706ade,326f94d6cfb1bf25,2019-10-18,6.466667
0019ec81fd706ade85073dbd1be65ed9,0019ec81fd706ade,85073dbd1be65ed9,2019-10-18,7.000000
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,2.550000
...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0.000000
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,14.522293
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,7.643312


### Remove debut fights
There isn't any historical data for fighters with debut fights, so for now we will not use them in our analysis.

In [67]:
fighter_bout_inst = functions.remove_debut_bouts(fighter_bout_inst)

In [68]:
fighter_bout_inst

,bout_id,fighter_id,date,sig_str_a_p1m
fighter_bout_inst,,,,
000da3152b7b5ab16da99156486ed6c2,000da3152b7b5ab1,6da99156486ed6c2,2006-07-08,5.866667
000da3152b7b5ab1d1a1314976c50bef,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,2.600000
0027e179b743c86c3aa794cbe1e3484b,0027e179b743c86c,3aa794cbe1e3484b,2015-03-14,2.550000
0027e179b743c86c91ea901c458e95dd,0027e179b743c86c,91ea901c458e95dd,2015-03-14,3.412500
002921976d27b7dab4ad3a06ee4d660c,002921976d27b7da,b4ad3a06ee4d660c,2014-12-13,5.928854
...,...,...,...,...
ffe629a5232a878bb361180739bed4b0,ffe629a5232a878b,b361180739bed4b0,2003-06-06,0.000000
ffea776913451b6d22a92d7f62195791,ffea776913451b6d,22a92d7f62195791,2015-02-28,14.522293
ffea776913451b6d75e5fec9f72910ef,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,7.643312


## Calculate metrics

The metrics I will using in this notebook are:
 - average successful significant strikes for each fighter (ASSS)
 - average significant strike accuracy (ASSA)
 - average significant strike defense (ASSD)

### Career Averages

In [69]:
list_of_metrics = ['sig_str_a_p1m',
       'td_s_p15m_di', 'sig_str_s_p1m_di', 'sig_str_a_p1m_di',
       'ground_s_p15m_di', 'ground_a_p15m_di', 'clinch_s_p15m_di',
       'clinch_a_p15m_di', 'distance_a_p1m_di', 'distance_s_p1m_di']

In [70]:
for metric in list_of_metrics:
    print(metric+'\n')
    values = fighter_bout_inst.apply(lambda row: functions.calculate_metric_average(metric, row['fighter_id'], row['date'], data), axis=1)
    fighter_bout_inst['ca_'+metric] = values
    values = fighter_bout_inst.apply(lambda row: functions.calculate_3_fight_average(metric, row['fighter_id'], row['date'], data), axis=1)
    fighter_bout_inst['3fa_'+metric] = values

sig_str_a_p1m



<ipython-input-70-2aa124c7a639>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['ca_'+metric] = values
<ipython-input-70-2aa124c7a639>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_bout_inst['3fa_'+metric] = values


td_s_p15m_di

sig_str_s_p1m_di

sig_str_a_p1m_di

ground_s_p15m_di

ground_a_p15m_di

clinch_s_p15m_di

clinch_a_p15m_di

distance_a_p1m_di

distance_s_p1m_di



### I need to fights between fighters who have less than three fights in the UFC

In [71]:
mask = fighter_bout_inst['3fa_sig_str_a_p1m'].notnull()
fighter_bout_inst_test = fighter_bout_inst[mask]

In [72]:
fighter_bout_inst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7984 entries, 000da3152b7b5ab16da99156486ed6c2 to fffa21388cdd78b7c80095f6092271a7
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   bout_id                7984 non-null   object        
 1   fighter_id             7984 non-null   object        
 2   date                   7984 non-null   datetime64[ns]
 3   sig_str_a_p1m          7984 non-null   float64       
 4   ca_sig_str_a_p1m       7959 non-null   float64       
 5   3fa_sig_str_a_p1m      7240 non-null   object        
 6   ca_td_s_p15m_di        7959 non-null   float64       
 7   3fa_td_s_p15m_di       7240 non-null   object        
 8   ca_sig_str_s_p1m_di    7959 non-null   float64       
 9   3fa_sig_str_s_p1m_di   7240 non-null   object        
 10  ca_sig_str_a_p1m_di    7959 non-null   float64       
 11  3fa_sig_str_a_p1m_di   7240 non-null   object        
 12  ca_groun

Here's how many fights i lose by removing all bouts between fighters with less than 3 fights.

In [73]:
len(fighter_bout_inst) - len(fighter_bout_inst_test)

744

This is how much of a percentage of my data that I'm losing.

In [74]:
((len(fighter_bout_inst) - len(fighter_bout_inst_test))/len(fighter_bout_inst))*100

9.318637274549097

In [75]:
fighter_bout_inst = fighter_bout_inst_test

## Create the final dataframe

First I will get a list af all bout ids. Then I will create a dataframe with the first row and another dataframe with the second row. Then I will join those dataframes along the column axis.

In [76]:
model_df = functions.merge_fighter_instances(fighter_bout_inst)

/home/max/sports_science/match_finder/src/functions.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_0 = list(instances_df.groupby(key).inst_id.max())


## Creating tsss_bout

tsss_bout: Total Successful Significant Strikes for the Bout. This metric measure the combined total number of significant strikes in a bout.

In [77]:
model_df['c_sig_str_a_p1m'] = model_df['sig_str_a_p1m_0'] + model_df['sig_str_a_p1m_1']
model_df

,bout_id,fighter_id_0,date_0,sig_str_a_p1m_0,ca_sig_str_a_p1m_0,3fa_sig_str_a_p1m_0,ca_td_s_p15m_di_0,3fa_td_s_p15m_di_0,ca_sig_str_s_p1m_di_0,3fa_sig_str_s_p1m_di_0,...,ca_clinch_s_p15m_di_1,3fa_clinch_s_p15m_di_1,ca_clinch_a_p15m_di_1,3fa_clinch_a_p15m_di_1,ca_distance_a_p1m_di_1,3fa_distance_a_p1m_di_1,ca_distance_s_p1m_di_1,3fa_distance_s_p1m_di_1,inst_id_1,c_sig_str_a_p1m
0,000da3152b7b5ab1,d1a1314976c50bef,2006-07-08,2.600000,5.455385,5.45538,-5.681781,-5.68178,0.104453,0.104453,...,4.595190,4.59519,0.080038,0.0800381,0.366525,0.366525,1.010532,1.01053,000da3152b7b5ab16da99156486ed6c2,8.466667
1,0027e179b743c86c,91ea901c458e95dd,2015-03-14,3.412500,5.235224,6.02175,1.500000,0.857143,1.599104,1.59872,...,-5.000000,-5,-3.000000,-3,-5.733333,-5.73333,-2.533333,-2.53333,0027e179b743c86c3aa794cbe1e3484b,5.962500
2,002921976d27b7da,ebc1f40e00e0c481,2014-12-13,1.185771,8.788065,10.8811,-0.433461,-0.6,-0.916931,0.864547,...,3.825798,-15.2929,-7.405384,-35.4358,-3.095198,-2.87071,-1.265419,-1.43001,002921976d27b7dab4ad3a06ee4d660c,7.114625
3,002c1562708ac307,44470bfd9483c7ad,2014-05-24,10.731707,3.200000,3.2,3.000000,3,0.466667,0.466667,...,2.628510,-1.00714,3.107143,-0.407143,1.837749,1.71,0.249837,-0.0128571,002c1562708ac30722a92d7f62195791,27.073171
4,002cb1bb411c5f60,d897897060f10a3a,2006-03-04,9.760000,11.630700,8.20843,-0.307581,1.03443,4.098703,3.31271,...,-10.989634,-12.2967,-9.656301,-10.5824,-0.024032,0.111959,-0.184414,-0.179961,002cb1bb411c5f6022e47b53e4ceb27c,12.640000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,ff64fc34065565d0,6fb1ba67bef41b37,2015-05-30,6.133333,6.486257,6.44444,1.429825,-0.333333,0.573392,0.955556,...,5.199375,6.25735,6.996808,6.25735,-0.600820,0.109339,-0.009273,0.565661,ff64fc34065565d007225ba28ae309b6,13.923810
3312,ff872fa3e9ec32a9,b7d524c77c27389b,2008-06-07,3.666667,6.067757,5.89327,0.779086,-1.89989,0.975783,0.653265,...,5.219740,4.57132,7.296663,5.77132,4.438256,2.2631,2.433883,0.651732,ff872fa3e9ec32a99fe85152f351e737,9.400000
3313,ffd3e3d37cba32da,92a9aa9c93192871,2014-10-25,10.266667,9.590387,5.06667,3.018444,2.66667,0.320330,1.48889,...,-2.625000,-3,-3.750000,-4.28571,0.427614,0.759949,-0.747097,-0.419829,ffd3e3d37cba32da7413b80dbb0f8f9f,15.733333
3314,ffea776913451b6d,75e5fec9f72910ef,2015-02-28,7.643312,5.866690,4.37778,3.846692,5.33333,-0.431887,-0.711111,...,1.852669,0.540941,1.276528,1.54094,1.106705,5.03862,0.508914,2.59808,ffea776913451b6d22a92d7f62195791,22.165605


In [78]:
model_df.columns

Index(['bout_id', 'fighter_id_0', 'date_0', 'sig_str_a_p1m_0',
       'ca_sig_str_a_p1m_0', '3fa_sig_str_a_p1m_0', 'ca_td_s_p15m_di_0',
       '3fa_td_s_p15m_di_0', 'ca_sig_str_s_p1m_di_0', '3fa_sig_str_s_p1m_di_0',
       'ca_sig_str_a_p1m_di_0', '3fa_sig_str_a_p1m_di_0',
       'ca_ground_s_p15m_di_0', '3fa_ground_s_p15m_di_0',
       'ca_ground_a_p15m_di_0', '3fa_ground_a_p15m_di_0',
       'ca_clinch_s_p15m_di_0', '3fa_clinch_s_p15m_di_0',
       'ca_clinch_a_p15m_di_0', '3fa_clinch_a_p15m_di_0',
       'ca_distance_a_p1m_di_0', '3fa_distance_a_p1m_di_0',
       'ca_distance_s_p1m_di_0', '3fa_distance_s_p1m_di_0', 'inst_id_0',
       'fighter_id_1', 'date_1', 'sig_str_a_p1m_1', 'ca_sig_str_a_p1m_1',
       '3fa_sig_str_a_p1m_1', 'ca_td_s_p15m_di_1', '3fa_td_s_p15m_di_1',
       'ca_sig_str_s_p1m_di_1', '3fa_sig_str_s_p1m_di_1',
       'ca_sig_str_a_p1m_di_1', '3fa_sig_str_a_p1m_di_1',
       'ca_ground_s_p15m_di_1', '3fa_ground_s_p15m_di_1',
       'ca_ground_a_p15m_di_1', '3fa_gro

In [79]:
model_df = model_df.loc[:,['ca_sig_str_a_p1m_0', '3fa_sig_str_a_p1m_0', 'ca_td_s_p15m_di_0',
                           '3fa_td_s_p15m_di_0', 'ca_sig_str_s_p1m_di_0', '3fa_sig_str_s_p1m_di_0',
                           'ca_sig_str_a_p1m_di_0', '3fa_sig_str_a_p1m_di_0',
                           'ca_ground_s_p15m_di_0', '3fa_ground_s_p15m_di_0',
                           'ca_ground_a_p15m_di_0', '3fa_ground_a_p15m_di_0',
                           'ca_clinch_s_p15m_di_0', '3fa_clinch_s_p15m_di_0',
                           'ca_clinch_a_p15m_di_0', '3fa_clinch_a_p15m_di_0',
                           'ca_distance_a_p1m_di_0', '3fa_distance_a_p1m_di_0',
                           'ca_distance_s_p1m_di_0', '3fa_distance_s_p1m_di_0',
                           'ca_sig_str_a_p1m_1',
                           '3fa_sig_str_a_p1m_1', 'ca_td_s_p15m_di_1', '3fa_td_s_p15m_di_1',
                           'ca_sig_str_s_p1m_di_1', '3fa_sig_str_s_p1m_di_1',
                           'ca_sig_str_a_p1m_di_1', '3fa_sig_str_a_p1m_di_1',
                           'ca_ground_s_p15m_di_1', '3fa_ground_s_p15m_di_1',
                           'ca_ground_a_p15m_di_1', '3fa_ground_a_p15m_di_1',
                           'ca_clinch_s_p15m_di_1', '3fa_clinch_s_p15m_di_1',
                           'ca_clinch_a_p15m_di_1', '3fa_clinch_a_p15m_di_1',
                           'ca_distance_a_p1m_di_1', '3fa_distance_a_p1m_di_1',
                           'ca_distance_s_p1m_di_1', '3fa_distance_s_p1m_di_1',
                           'c_sig_str_a_p1m']]

In [80]:
model_df

,ca_sig_str_a_p1m_0,3fa_sig_str_a_p1m_0,ca_td_s_p15m_di_0,3fa_td_s_p15m_di_0,ca_sig_str_s_p1m_di_0,3fa_sig_str_s_p1m_di_0,ca_sig_str_a_p1m_di_0,3fa_sig_str_a_p1m_di_0,ca_ground_s_p15m_di_0,3fa_ground_s_p15m_di_0,...,3fa_ground_a_p15m_di_1,ca_clinch_s_p15m_di_1,3fa_clinch_s_p15m_di_1,ca_clinch_a_p15m_di_1,3fa_clinch_a_p15m_di_1,ca_distance_a_p1m_di_1,3fa_distance_a_p1m_di_1,ca_distance_s_p1m_di_1,3fa_distance_s_p1m_di_1,c_sig_str_a_p1m
0,5.455385,5.45538,-5.681781,-5.68178,0.104453,0.104453,0.482591,0.482591,12.923077,12.9231,...,18.5292,4.595190,4.59519,0.080038,0.0800381,0.366525,0.366525,1.010532,1.01053,8.466667
1,5.235224,6.02175,1.500000,0.857143,1.599104,1.59872,2.388358,2.38337,13.204478,6.86354,...,0,-5.000000,-5,-3.000000,-3,-5.733333,-5.73333,-2.533333,-2.53333,5.962500
2,8.788065,10.8811,-0.433461,-0.6,-0.916931,0.864547,-1.198873,3.37951,-1.755793,-0.0156793,...,24.518,3.825798,-15.2929,-7.405384,-35.4358,-3.095198,-2.87071,-1.265419,-1.43001,7.114625
3,3.200000,3.2,3.000000,3,0.466667,0.466667,-1.000000,-1,3.000000,3,...,1.60714,2.628510,-1.00714,3.107143,-0.407143,1.837749,1.71,0.249837,-0.0128571,27.073171
4,11.630700,8.20843,-0.307581,1.03443,4.098703,3.31271,7.565028,4.65123,17.180382,18.908,...,19.1461,-10.989634,-12.2967,-9.656301,-10.5824,-0.024032,0.111959,-0.184414,-0.179961,12.640000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,6.486257,6.44444,1.429825,-0.333333,0.573392,0.955556,1.119006,0.755556,5.789474,10,...,0.0808824,5.199375,6.25735,6.996808,6.25735,-0.600820,0.109339,-0.009273,0.565661,13.923810
3312,6.067757,5.89327,0.779086,-1.89989,0.975783,0.653265,0.286456,0.479861,6.327674,-0.880148,...,-1.8,5.219740,4.57132,7.296663,5.77132,4.438256,2.2631,2.433883,0.651732,9.400000
3313,9.590387,5.06667,3.018444,2.66667,0.320330,1.48889,4.057003,2.55556,6.073953,18.6667,...,23.8957,-2.625000,-3,-3.750000,-4.28571,0.427614,0.759949,-0.747097,-0.419829,15.733333
3314,5.866690,4.37778,3.846692,5.33333,-0.431887,-0.711111,-0.736226,-1.4,1.829027,4.66667,...,2.67857,1.852669,0.540941,1.276528,1.54094,1.106705,5.03862,0.508914,2.59808,22.165605


In [81]:
model_df.to_csv('../../data/model_7_data.csv')